In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import json
from collections import defaultdict
import utils

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [30]:
events = pd.read_json('events\events_World_Cup.json')
matches = pd.read_json('matches\matches_World_Cup.json')
teams = pd.read_json('teams.json')


In [3]:
#tira penalidades
events = events[events['matchPeriod'] != 'P']

In [24]:
matches.head()

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,groupName,referees,duration,competitionId
0,Played,4165368,0,"{'9598': {'scoreET': 0, 'coachId': 122788, 'si...",10078,2018-07-15 15:00:00,4418,Olimpiyskiy stadion Luzhniki,2058017,"France - Croatia, 4 - 2",2018-07-15 17:00:00-02:00,,"[{'refereeId': 378051, 'role': 'referee'}, {'r...",Regular,28
1,Played,4165367,0,"{'2413': {'scoreET': 0, 'coachId': 276858, 'si...",10078,2018-07-14 14:00:00,5629,Stadion Krestovskyi,2058016,"Belgium - England, 2 - 0",2018-07-14 16:00:00-02:00,,"[{'refereeId': 380599, 'role': 'referee'}, {'r...",Regular,28
2,Played,4165366,0,"{'2413': {'scoreET': 1, 'coachId': 276858, 'si...",10078,2018-07-11 18:00:00,9598,Olimpiyskiy stadion Luzhniki,2058015,"Croatia - England, 2 - 1 (E)",2018-07-11 20:00:00-02:00,,"[{'refereeId': 384995, 'role': 'referee'}, {'r...",ExtraTime,28
3,Played,4165366,0,"{'5629': {'scoreET': 0, 'coachId': 8605, 'side...",10078,2018-07-10 18:00:00,4418,Stadion Krestovskyi,2058014,"France - Belgium, 1 - 0",2018-07-10 20:00:00-02:00,,"[{'refereeId': 378232, 'role': 'referee'}, {'r...",Regular,28
4,Played,4165365,0,"{'14358': {'scoreET': 2, 'coachId': 264893, 's...",10078,2018-07-07 18:00:00,9598,Olimpiyskiy Stadion Fisht,2058012,"Russia - Croatia, 2 - 2 (P)",2018-07-07 20:00:00-02:00,,"[{'refereeId': 378204, 'role': 'referee'}, {'r...",Penalties,28


In [25]:
#fazer o mesmo do df das networks para esse aqui (criar df com features)

df = matches[['wyId']]
df = df.rename(columns={'wyId':'matchID'})

#colocar team1 e team2
home_l = []
away_l = []

for match in matches['teamsData']:
    t0 = list(match.values())[0]
    t1 = list(match.values())[1]

    if t0['side'] == 'home':
        home_l.append(t0['teamId'])
        away_l.append(t1['teamId'])  
    else: 
        away_l.append(t0['teamId'])
        home_l.append(t1['teamId'])

df['team1_ID'] = home_l
df['team2_ID'] = away_l

df['date'] = matches['date']

df.head()

,matchID,team1_ID,team2_ID,date
0,2058017,4418,9598,2018-07-15 17:00:00-02:00
1,2058016,5629,2413,2018-07-14 16:00:00-02:00
2,2058015,9598,2413,2018-07-11 20:00:00-02:00
3,2058014,4418,5629,2018-07-10 20:00:00-02:00
4,2058012,14358,9598,2018-07-07 20:00:00-02:00


In [5]:
# adaptando coluna 'tags' para conseguir interpretar se evento foi bem sucedido ou não. 'id' = 1801 ou 1802
events['tags'] = events['tags'].map(utils.converting_tags)

In [6]:
# GK_SAVES

#contar quantos subEvents = 'Save attempt' com tag = 1801(bem sucedido) para todos os (match, team)

gk_saves = events[(events['eventName'] == 'Save attempt') & (events['tags'].apply(lambda x: 1801 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
gk_saves.columns = gk_saves.columns.droplevel(0)
gk_saves = gk_saves.reset_index().rename(columns={'count':'gk_saves'})
gk_saves.head()

#juntar feature com df
df = df.merge(gk_saves, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(gk_saves, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"gk_saves_x": "gk_saves_T1", "gk_saves_y": "gk_saves_T2"})

df = df.fillna(0)
#df

In [7]:
#RED_CARDS

#contar quantos subEvents = 'Foul' com tag = 1701(red card) ou 1703(second yellow card) para todos os (match, team)

red_card = events[events['tags'].apply(lambda x: (1701 in x) or (1703 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
red_card.columns = red_card.columns.droplevel(0)
red_card = red_card.reset_index().rename(columns={'count':'red_card'})

#juntar feature com df
df = df.merge(red_card, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(red_card, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"red_card_x": "red_card_T1", "red_card_y": "red_card_T2"})

df = df.fillna(0)
#df


In [8]:
#YELLOW_CARDS

#contar quantos subEvents = 'Foul' com tag = 1702(yellow card) ou 1703(second yellow card) para todos os (match, team)

yellow_card = events[events['tags'].apply(lambda x: (1702 in x) or (1703 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
yellow_card.columns = yellow_card.columns.droplevel(0)
yellow_card = yellow_card.reset_index().rename(columns={'count':'yellow_card'})

#juntar feature com df
df = df.merge(yellow_card, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(yellow_card, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"yellow_card_x": "yellow_card_T1", "yellow_card_y": "yellow_card_T2"})

df = df.fillna(0)
#df

In [9]:
#ASSISTS

assists = events[events['tags'].apply(lambda x: (301 in x))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
assists.columns = assists.columns.droplevel(0)
assists = assists.reset_index().rename(columns={'count':'assists'})

#juntar feature com df
df = df.merge(assists, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(assists, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"assists_x": "assists_T1", "assists_y": "assists_T2"})

df = df.fillna(0)
#df

In [10]:
#FINALIZAÇÕES

#contar quantos subEventName = 'Shot', 'Free kick shot' ou 'Penalty' para todos os (match, team)

shots = events[events['subEventName'].isin(['Shot', 'Free kick shot', 'Penalty'])].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
shots.columns = shots.columns.droplevel(0)
shots = shots.reset_index().rename(columns={'count':'shots'})

#juntar feature com df
df = df.merge(shots, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(shots, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"shots_x": "shots_T1", "shots_y": "shots_T2"})

df = df.fillna(0)
#df

In [11]:
#CHUTES A GOL

#contar quantos subEventName = 'Shot', 'Free kick shot' ou 'Penalty' com tag = 1801(accurate) para todos os (match, team)

shots_on_target = events[(events['subEventName'].isin(['Shot', 'Free kick shot', 'Penalty'])) & (events['tags'].apply(lambda x: (1801 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
shots_on_target.columns = shots_on_target.columns.droplevel(0)
shots_on_target = shots_on_target.reset_index().rename(columns={'count':'shots_on_target'})

#juntar feature com df
df = df.merge(shots_on_target, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(shots_on_target, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"shots_on_target_x": "shots_on_target_T1", "shots_on_target_y": "shots_on_target_T2"})

df = df.fillna(0)
#df

In [12]:
# *** ARRUMAR GOL CONTRA ***

#GOLS

goals = events[(events['eventName'] != 'Save attempt') & (events['tags'].apply(lambda x: (101 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
goals.columns = goals.columns.droplevel(0)
goals = goals.reset_index().rename(columns={'count':'goals'})

own_goals = events[(events['eventName'] != 'Save attempt') & (events['tags'].apply(lambda x: (102 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
own_goals.columns = own_goals.columns.droplevel(0)
own_goals = own_goals.reset_index().rename(columns={'count':'own_goals'})

#juntar feature com df
df = df.merge(goals, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(goals, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.merge(own_goals, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(own_goals, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"goals_x": "goals_T1", "goals_y": "goals_T2"})
df = df.rename(columns={"own_goals_x": "own_goals_T1", "own_goals_y": "own_goals_T2"})

df = df.fillna(0)

df['goals_T1'] = df['goals_T1'] + df['own_goals_T2']
df['goals_T2'] = df['goals_T2'] + df['own_goals_T1']

df = df.drop(['own_goals_T1', 'own_goals_T2'], axis=1)
#df

In [13]:
#QUANTIDADE DE PASSES


passes = events[(events['eventName'] == 'Pass') & (events['tags'].apply(lambda x: (1801 in x)))].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
passes.columns = passes.columns.droplevel(0)
passes = passes.reset_index().rename(columns={'count':'passes'})

#juntar feature com df
df = df.merge(passes, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(passes, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"passes_x": "passes_T1", "passes_y": "passes_T2"})

df = df.fillna(0)
#df

In [14]:
#POSSE DE BOLA

#passes dados / total de passes na partida
df['ball_possession_T1'] = df['passes_T1'] / (df['passes_T1'] + df['passes_T2'])
df['ball_possession_T2'] = df['passes_T2'] / (df['passes_T1'] + df['passes_T2'])

#df

In [15]:
#ACCURACY

# **pass_accuracy**
total_passes = events[events['eventName'] == 'Pass'].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
total_passes.columns = total_passes.columns.droplevel(0)
total_passes = total_passes.reset_index().rename(columns={'count':'total_passes'})

#juntar feature com df
df = df.merge(total_passes, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(total_passes, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"total_passes_x": "total_passes_T1", "total_passes_y": "total_passes_T2"})

df['pass_acc_T1'] = df['passes_T1'] / df['total_passes_T1']
df['pass_acc_T2'] = df['passes_T2'] / df['total_passes_T2']



# **gk_accuracy**
gk_att = events[(events['eventName'] == 'Save attempt')].groupby(['matchId', 'teamId']).agg({'eventId':['count']})
gk_att.columns = gk_att.columns.droplevel(0)
gk_att = gk_att.reset_index().rename(columns={'count':'gk_att'})

#juntar feature com df
df = df.merge(gk_att, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)
df = df.merge(gk_att, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

df = df.rename(columns={"gk_att_x": "gk_att_T1", "gk_att_y": "gk_att_T2"})

df['gk_acc_T1'] = df['gk_saves_T1'] / df['gk_att_T1']
df['gk_acc_T2'] = df['gk_saves_T2'] / df['gk_att_T2']



# **shot_accuracy**
df['shot_acc_T1'] = df['shots_on_target_T1'] / df['shots_T1']
df['shot_acc_T2'] = df['shots_on_target_T2'] / df['shots_T2']

df = df.drop(['total_passes_T1', 'total_passes_T2', 'gk_att_T1', 'gk_att_T2'], axis=1)
df = df.fillna(0)
#df

In [16]:
#definindo vencedor e retirando empates
df['winner'] = matches['winner']
df = df.loc[df['winner'] != 0]
matches = matches.loc[matches['winner'] != 0]
df['winner'] = np.where((matches['winner'] == df['team1_ID']), 1, 0)

In [18]:
df.shape

(55, 28)